In [ ]:
import __paths__
from os.path import join as jj

import IPython
from bokeh.io import output_notebook
from IPython.display import display
from penelope import topic_modelling as tm
from penelope import utility as pu
from penelope.notebook import topic_modelling as ntm

import westac.riksprot.parlaclarin.metadata as md
import westac.riksprot.parlaclarin.speech_text as sr
from notebooks.riksdagens_protokoll import topic_modeling as wtm  # pylint: disable=unused-import

IPython.Application.instance().kernel.do_shutdown(True)


output_notebook(hide_banner=True)
pu.set_default_options()

model_folder: str = jj(__paths__.data_folder, "riksdagen_corpus_data/tm_1920-2020_500-TF5-MP0.02.500000.lemma.mallet")
riksprot_metadata: md.IRiksprotMetaData = md.IRiksprotMetaData.load(database_filename="./data/riksprot_metadata.db")
speech_repository: sr.SpeechTextRepository = sr.SpeechTextRepository(
    source=jj(__paths__.data_folder, "riksdagen_corpus_data/tagged_frames_v0.3.0_20201218"),
    riksprot_metadata=riksprot_metadata,
)
# FIXME #166 Load fails if topic models if topic model is slimmed-
inferred_topics: tm.InferredTopicsData = tm.InferredTopicsData.load(
    # folder=jj(__paths__.data_folder, "riksdagen_corpus_data/tm_1920-2020_500-topics-mallet/"),
    folder=model_folder,
    slim=True,
)
inferred_topics.document_index = riksprot_metadata.overload_by_member_data(
    inferred_topics.document_index, encoded=True, drop=True
)
state = dict(inferred_topics=inferred_topics)
# ui = wtm.RiksprotLoadGUI(riksprot_metadata, corpus_folder="/data/westac/riksdagen_corpus_data/", corpus_config=None,state=state, slim=True).setup()
# ui = ntm.WordcloudGUI(state).setup()
# ui = wtm.RiksprotFindTopicDocumentsGUI(riksprot_metadata, speech_repository=speech_repository, state=state).setup()
# ui = wtm.RiksprotBrowseTopicDocumentsGUI(riksprot_metadata, speech_repository=speech_repository, state=state).setup()
ui = ntm.TopicWordDistributionGUI(state=state).setup()
# ui = wtm.RiksprotTopicTrendsGUI(riksprot_metadata, speech_repository=speech_repository, state=state).setup()
# ui = ntm.TopicOverviewGUI(state=state).setup()
# ui = wtm.RiksprotTopicTrendsOverviewGUI(riksprot_metadata, speech_repository=speech_repository, state=state).setup()
# ui = ntm.TopicTrendsGUI(state=state).setup()
# ui = wtm.RiksprotTopicTrendsGUI(riksprot_metadata, speech_repository=speech_repository, state=state).setup()
# ui = ntm.TopicTopicGUI(state=state).setup()
# ui = wtm.RiksprotTopicTopicGUI(riksprot_metadata, speech_repository=speech_repository, state=state).setup()
# ui = ntm.PivotTopicNetworkGUI(pivot_key_specs=riksprot_metadata.member_property_specs, state=state).setup()
# ui = wtm.RiksprotTopicTopicGUI(riksprot_metadata, speech_repository=speech_repository, state=state).setup()
# ui = ntm.EditTopicLabelsGUI(folder=model_folder, state=state).setup()
# ui: ntm.DefaultTopicDocumentNetworkGui = ntm.DefaultTopicDocumentNetworkGui(
#     pivot_key_specs=riksprot_metadata.member_property_specs, state=state
# ).setup()
# ui: ntm.FocusTopicDocumentNetworkGui = ntm.FocusTopicDocumentNetworkGui(
#     pivot_key_specs=riksprot_metadata.member_property_specs, state=state
# ).setup()
# ui._find_text.value = "byggnad"
display(ui.layout())
ui.update_handler()